In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('D:\\Datasets\\Recommender_system\\movies.csv')

In [50]:
rating =  pd.read_csv('D:\\Datasets\\Recommender_system\\ratings.csv')

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
rating.head()
# rating.info()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [6]:
# movie_name = movies.title.tolist()

In [7]:
# no_of_user = rating.userId.unique()

In [8]:
# l = []
# for user in no_of_user[11:50]:
#     print(user)
#     movie_id = rating.loc[rating.userId ==user, 'movieId']
#     movie_rate = {}
#     for each_movie in movie_id:
#         movie_title = movies.loc[movies.movieId == each_movie, 'title'].tolist()[0]
#         movie_rate['user_id'] = user
#         movie_rate[movie_title] = rating.loc[(rating.userId ==user) & (rating.movieId==each_movie), 'rating'].tolist()[0]
        
#     l.append(movie_rate)

In [9]:
# coll_filt_df = pd.DataFrame(data=l)

In [10]:
# coll_filt_df.set_index(keys='user_id', inplace=True)

In [11]:
# coll_filt_df.to_csv("D:\\Datasets\\Recommender_system\\Uer_movies_rating.csv", mode='a', header=False)

In [12]:
user_movie_rating = pd.merge(rating, movies)

In [13]:
user_movie_rating.head()

,userId,movieId,rating,timestamp,title,genres
0,1,169,2.5,1204927694,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama
1,13,169,1.0,974868393,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama
2,14,169,3.0,845470321,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama
3,17,169,1.0,944991371,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama
4,68,169,1.0,1011092044,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama


In [14]:
user_movie_rating.drop(labels=['genres', 'timestamp'], axis= 1, inplace=True)

In [15]:
user_movie_rating.head()

,userId,movieId,rating,title
0,1,169,2.5,Free Willy 2: The Adventure Home (1995)
1,13,169,1.0,Free Willy 2: The Adventure Home (1995)
2,14,169,3.0,Free Willy 2: The Adventure Home (1995)
3,17,169,1.0,Free Willy 2: The Adventure Home (1995)
4,68,169,1.0,Free Willy 2: The Adventure Home (1995)


### Due to memory reason we will create pivot table first some rows

In [16]:
df = user_movie_rating[:500000].pivot_table(index=['userId'], columns='movieId', values='rating')

In [17]:
df.head()

movieId,16,39,45,47,94,101,169,246,288,296,...,377,2394,2431,2445,2471,2571,48516,109487,112552,112556
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,NaN,5.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.5,NaN,4.0,5.0,4.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,4.0,4.0,2.0,5.0,4.0,NaN,4.0,2.0,4.0,...,4.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,4.5,4.5


In [18]:
df.columns

Int64Index([    16,     39,     45,     47,     94,    101,    169,    246,
               288,    296,    345,    356,    357,    377,   2394,   2431,
              2445,   2471,   2571,  48516, 109487, 112552, 112556],
           dtype='int64', name='movieId')

In [19]:
df.shape

(143574, 23)

In [20]:
df.dropna(axis=1, thresh=50, inplace=True)  ## drop that movie which don't have minimum 50 user's rating.

In [21]:
df.fillna(value=0, inplace=True)

In [22]:
similarty_df = df.corr()                ### use Pearson Correlation for similarty between movies.

In [44]:
def similar_movies(movie_id, rating_of_movie):
    similar_score = similarty_df[movie_id]*(rating_of_movie-2.5)      ### -2.5 for making less rated movie by user make them less weighted
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score

In [39]:
user_id = np.random.randint(low=1, high=10, size=1)[0]
user_id

5

In [45]:
user_viewed_movies = rating[rating.userId == user_id][['movieId', 'rating']]
user_viewed_movies

AttributeError: 'float' object has no attribute 'userId'

In [46]:
Recommended_movies = pd.DataFrame()
for i in user_viewed_movies.itertuples():
    movieid = i[1]
    user_rating = i[2]
    Recommended_movies  =Recommended_movies.append(similar_movies(movieid, user_rating))

Recommended_movies

,16,39,45,47,94,101,169,246,288,296,...,377,2394,2431,2445,2471,2571,48516,109487,112552,112556
169,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
2471,0.041277,0.041788,0.031649,0.029889,0.021105,0.026874,0.032438,0.010698,0.033158,0.011326,...,0.034845,0.051003,0.078700,0.03907,0.500000,0.045612,0.017713,-0.008371,-0.006914,-0.003599
48516,0.340310,-0.027311,-0.073964,0.626169,-0.014162,0.194340,-0.028855,-0.066854,0.107555,0.456773,...,0.074815,0.030969,0.145781,0.03530,0.088565,0.672856,2.500000,0.538681,0.467064,0.558096


In [47]:
Recommended_movies = Recommended_movies.sum(axis=0).sort_values(ascending=False )[:10]

In [48]:
Recommended_movies

48516     2.517713
2571      0.718467
47        0.656058
2471      0.588565
112556    0.554497
109487    0.530310
296       0.468099
112552    0.460150
16        0.381588
356       0.340107
dtype: float64

5.0